In [17]:
%load_ext autoreload
%autoreload 2

import sys
import os 
import pandas as pd
import numpy as np

import soepy

from dev_library import plot_basics_choices
from dev_library import plot_basics_wages
from dev_library import *
import pandas as pd
pd.set_option('display.max_rows', 500)
sys.path.insert(0, os.environ["PROJECT_DIR"] + "/estimations/basecamp")
os.chdir(os.environ["PROJECT_DIR"] + "/estimations/basecamp")
from moments import get_moments

In [9]:
fname = os.environ["PROJECT_DIR"] + "/resources/soepcore_struct_prep.dta"
df_obs = pd.read_stata(fname, convert_categoricals=False)
df_obs = df_alignment(df_obs)

params_start = pd.read_pickle("start.soepy.pkl")
fname  = "resources/model_spec_init.yml"
df_sim = df_alignment(soepy.simulate(params_start, fname))

In [43]:
_ = get_moments(df_obs)
df_obs_int = pd.read_pickle("df-debug.pkl")

In [48]:
_ = get_moments(df_sim)
df_sim_int = pd.read_pickle("df-debug.pkl")

In [57]:
df_sim_int.sort_index()["Value"].to_list()

[0.06897463002114165,
 0.8847780126849895,
 0.046247357293868925,
 0.050256410256410255,
 0.8246153846153846,
 0.12512820512820513,
 0.0,
 0.0,
 0.0,
 0.07827458047077374,
 0.8440139655366595,
 0.07771145399256674,
 0.05137728257505416,
 0.7833488084184463,
 0.16527390900649955,
 0.0,
 0.0,
 0.0,
 0.07588532883642496,
 0.8464305789769534,
 0.0776840921866217,
 0.0619074177356386,
 0.724484104852203,
 0.21360847741215838,
 0.0468384074941452,
 0.6416861826697893,
 0.3114754098360656,
 0.07649369443870167,
 0.8487698986975398,
 0.07473640686375853,
 0.07031121356825293,
 0.6769306767660135,
 0.2527581096657336,
 0.049075844486934354,
 0.6195028680688337,
 0.331421287444232,
 0.07122972064893923,
 0.8517807430162235,
 0.07698953633483728,
 0.0713165026387106,
 0.6120382256454143,
 0.31664527171587503,
 0.035516969218626675,
 0.6282557221783741,
 0.3362273086029992,
 0.059599676869221796,
 0.867516380935284,
 0.07288394219549411,
 0.07506123037026365,
 0.5710992652355569,
 0.35383950439417